# Effective C++ Notes

And so begin my notes primarily based on the Effective C++ books by Scott Meyers (and various other sources on the interwebs). This is in an attempt to improve how I write my C++ code (which I tend to write quite a bit of) and incorporate better (and hopefully safer and more readable). 

Also, pointers make me a bit nervous.

Let's go ahead and include all the libraries we need beforehand

In [1]:
#include <iostream>

### 1. Try not to use macro definitions

C code often contains macro definitions for constants and expressions to make the code more readable and easier to use, and this is not uncommon in C++ code either. You can look up more about macros here- https://gcc.gnu.org/onlinedocs/cpp/Macros.html#Macros

In [2]:
#define PIE     3.14                    // You can now use PIE in place of 3.14 wherever the definition applies

#define SQR(a)  ((a) * (a))             // You can now use SQR to find the square of a number a

int radius = 3;
double area = PIE * SQR(radius);

std::cout << "The area of a circle with radius " << radius
          << " is " << area << "\n";

The area of a circle with radius 3 is 28.26


However, there are some issues with macros, for which we need to understand how they work. Long story short, before the compiler compiles the code, a preprocessor takes a look at the code and replaces all occurances of macros with their values. For instance, running the above code with g++'s preprocessor yields the following:

In [3]:
!gcc -E macro-example.cpp | sed -ne '/^int main() {$/,$ p' # This prints out preprocessed C++ code

int main() {
 int radius = 3;
 double area = 3.14 * ((radius) * (radius));

 std::cout << "The area of a circle with radius " << radius
           << " is " << area;

 return 0;
}


Because what preprocessors do is essentially replace the macro with the corresponding problem, it may lead to several issues. Primarily, depending on the compiler used, the macro symbol may not be tracked by the compiler and any compilation error involving `PIE` may end up referring to `3.14` rather than `PIE`. While this isn't a problem with modern compilers, it is still best to take this into consideration. A simple fix for this is to use `const double` as a global variable, or using the `static` keyword for class-specific constants which must be shared across all object instantiations.

In [4]:
const double GLOBAL_PIE = 3.14;

In [5]:
class REAL_CONSTS {
private:
    static constexpr double CLASS_PIE = 3.14;
};

This also allows for exploiting other features of C++ like encapsulation, which are not available for macros.

Macro functions tend to cause more substantial problems. For example, if the argument contains an increment operator like so:

In [6]:
radius = 3;
double buggy_area = PIE * SQR(++radius);

std::cout << "The area of a circle with radius " << radius
          << " is " << buggy_area << "\n";

input_line_11:3:31: warning: multiple unsequenced modifications to 'radius' [-Wunsequenced]
double buggy_area = PIE * SQR(++radius);
                              ^~
input_line_8:3:19: note: expanded from macro 'SQR'
#define SQR(a)  ((a) * (a))             // You can now use SQR to find the square of a number a
                  ^     ~


The area of a circle with radius 5 is 62.8


The output should have been "The area of a circle with radius 4 is 50.24", but since the expansion of `SQR(++radius)` is `((++radius) * (++radius))`, `radius` gets incremented twice, which is why its value is `5` and the product is `3.14 * 4 * 5 = 62.8`.

In [7]:
inline double SQR_FUN(double a) {
    return a * a;
}

radius = 3;
double correct_area = PIE * SQR_FUN(++radius);

std::cout << "The area of a circle with radius " << radius
          << " is " << correct_area << "\n";

The area of a circle with radius 4 is 50.24
